# Лабораторная работа №4

## "Численные методы решения задачи Коши"

У меня был 1 вариант заданий, следовательно использовались следующие данные:
1. Задача Коши
$$ u' = -u^2 + \frac{u}{x}, x \in [1, 2] $$
$$ u(1) = \frac{2}{3} $$
2. Точное решение $$ u(x) = \frac{2x}{x^2 + 2} $$
3. Методы:
- Неявный метод трапеций;
- Явный метод Рунге-Кутты 4-го порядка;
- Предиктор-корректорный метод Адамса 4-го порядка.

### Задание лабараторной работы

Решить задачу Коши для обыкновенного дифференциального уравнения первого порядка на отрезке $ [a, b] $ с шагом $ h = 0.1 $ методами, указанными в варианте задания. Оценить погрешность численного решения с шагом $ h = 0.1 $ с помощью правила Рунге (для одношаговых методов). Сравнить полученные численные решения с точным решением $ u(x) $. В одной системе координат построить график функции $ u(x) $ и график одного из полученных численных решений.\
\
По результатам лабораторной работы оформляется отчет. В содержание отчета должна быть включена следующая информация:
- Постановка задачи.
- Применяемые численные методы. Итерационный процесс метода Ньютона для реализации неявного метода трапеций.
- Правило Рунге оценки погрешности.
- Результаты вычислительного эксперимента, оформленные в виде таблицы.
- Выводы.
- Листинг программы с комментариями.

#### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

#### Задача Коши

In [4]:
def f(x, u):
    return -(u * u) + (u / x)

#### Производная

In [7]:
def derivative(x, u):
    return (1 / x) - (2 * u)

#### Точное решение

In [10]:
def solution(x):
    return (2 * x) / (x**2 + 2)

In [12]:
def correct_solution(a, b, h):
    n = int((b - a) / h)
    y = [0] * (n + 1)
    for i in range(n + 1):
        x_i = a + (h * i)
        y[i] = solution(x_i)
    return y

#### Оценки погрешности

Правило Рунге
$$ \frac{|y_{i, h} - y_{i, h/2}|}{2^p - 1} $$
$ p $ - порядок точности метода

In [16]:
def runge_rule(err, y_h, y_2h, p):
    return max(err, np.abs(y_2h - y_h) / ((2**p) - 1))

$$ max(|u(x_i) - y_i|)_{(i=0,N)} $$

In [19]:
def max_absolute_error(err, corr, y):
    return max(err, np.abs(corr - y))

#### Неявный метод трапеций

$$ y_{i + 1} = y_{i} + \frac{h}{2}(f(x_i, y_i) + f(x_{i + 1}, y_{i + 1}^k)) $$
Начальное приближение метода Ньютона:
$$ y_0 = u_0 $$
$$ y_{i + 1}^{0} = y_i $$
Итерационный процесс:
$$ y_{i + 1}^{k + 1} = y_{i + 1}^{k} - \frac{(y_{i + 1}^{k} - y_i - \frac{h}{2}(f(x_i, y_i) + f(x_{i + 1}, y_{i + 1}^{k})))}{1 - \frac{h}{2}f'(x_{i + 1}, y_{i + 1}^{k})} $$
Условие выхода из итерационного процесса:
$$ |y_{i + 1}^{k + 1} - y_{i + 1}^{k}| < EPS > 0 $$

In [23]:
def trapezoidal(a, b, h, u_0, epsilon=1e-7):
    n = int((b - a) / h)
    y = [0] * (n + 1)
    y[0] = u_0
    for i in range(n):
        x_i = a + (h * i)
        y_start = 0
        y_next = y[i]
        while np.abs(y_next - y_start) > epsilon:
            y_start = y_next
            y_next = y_start - ((y_start - y[i] - ((h / 2) * (f(x_i, y[i]) + f(x_i + h, y_start)))) / (1 - (h / 2) * derivative(x_i + h, y_start)))
        y[i + 1] = y_next
        y[i + 1] = y[i] + (h / 2) * (f(x_i, y[i]) + f(x_i + h, y[i + 1]))
    return y

#### Явный метод Рунге-Кутты 4-го порядка

$$ \left\{ \begin{array}{cl}
k_1 = f(x_i, y_i) \\
k_2 = f(x_i + \frac{h}{2}, y_i + \frac{hk_1}{2}) \\
k_3 = f(x_i + \frac{h}{2}, y_i + \frac{hk_2}{2}) \\
k_4 = f(x_i + h, y_i + hk_3) \\
y_{i + 1} = y_i + h(\frac{k_1}{6} + \frac{k_2}{3} + \frac{k_3}{3} + \frac{k_4}{6}) \\
\end{array} \right. $$

In [27]:
def runge_kutta(a, b, h, u_0):
    n = int((b - a) / h)
    y = [0] * (n + 1)
    y[0] = u_0
    for i in range(n):
        x_i = a + (h * i)
        k_1 = f(x_i, y[i])
        k_2 = f(x_i + (h / 2), y[i] + ((h * k_1) / 2))
        k_3 = f(x_i + (h / 2), y[i] + ((h * k_2) / 2))
        k_4 = f(x_i + h, y[i] + (h * k_3))
        y[i + 1] = y[i] + h * ((k_1 / 6) + (k_2 / 3) + (k_3 / 3) + (k_4 / 6))
    return y

#### Предиктор-корректорный метод Адамса 4-го порядка

$$ y_0 = u_0 $$
$$ y_1, y_2, y_3 \text{ - Runge-Kutta O}(h^4)$$
$$ y_{i + 1}^Я = y_i + \frac{h}{24}(55f_i - 59f_{i - 1} + 37f_{i - 2} - 9f_{i - 3}),\ \ \ \ \ f_i = f(x_i, y_i) $$
$$ y_{i + 1} = y_i + \frac{h}{24}(9f_{i + 1} + 19f_i - 5f_{i - 1} + f_{i - 2}),\ \ \ \ \ f_i = f(x_i, y_i^Я) $$

In [31]:
def adams(a, b, h, u_0):
    n = int((b - a) / h)
    y = [0] * (n + 1)
    y[0] = u_0
    for i in range(min(n, 3)):
        x_i = a + (h * i)
        k_1 = f(x_i, y[i])
        k_2 = f(x_i + (h / 2), y[i] + ((h * k_1) / 2))
        k_3 = f(x_i + (h / 2), y[i] + ((h * k_2) / 2))
        k_4 = f(x_i + h, y[i] + (h * k_3))
        y[i + 1] = y[i] + h * ((k_1 / 6) + (k_2 / 3) + (k_3 / 3) + (k_4 / 6))
    for i in range(3, n):
        x_i = a + (h * i)
        y_n = y[i] + (h / 24) * (55 * f(x_i, y[i]) - 59 * f(x_i - h, y[i - 1]) + 37 * f(x_i - (2 * h), y[i - 2]) - 9 * f(x_i - (3 * h), y[i - 3]))
        y[i + 1] = y[i] + (h / 24) * (9 * f(x_i + h, y_n) + 19 * f(x_i, y[i]) - 5 * f(x_i - h, y[i - 1]) + f(x_i - (2 * h), y[i - 2]))
    return y

#### Вызов функций

In [34]:
# Начальное значение u
u_0 = 2 / 3

# Значения a и b
a = 1
b = 2

# Значение шага
h = 0.1

# Создаём таблицу значений
df = pd.DataFrame(columns=["i",
                           "x",
                           "Точное значение",
                           "Неявный метод трапеций",
                           "Рунге-Кутты 4-го порядка",
                           "Адамса 4-го порядка"])
# Создаём таблицы ошибок
abs_err_df = pd.DataFrame(columns=["Неявный метод трапеций",
                           "Рунге-Кутты 4-го порядка",
                           "Адамса 4-го порядка"])

err_df = pd.DataFrame(columns=["Неявный метод трапеций",
                           "Рунге-Кутты 4-го порядка",
                           "Адамса 4-го порядка"])

# Вычисляем значения разных методов
y_h = [correct_solution(a, b, h), trapezoidal(a, b, h, u_0), runge_kutta(a, b, h, u_0), adams(a, b, h, u_0)]
y_2h = [trapezoidal(a, b, h * 2, u_0), runge_kutta(a, b, h * 2, u_0), adams(a, b, h * 2, u_0)]

# Вычисляем ошибки и заполняем таблицы
err = [0, 0, 0]
max_abs_err = [0, 0, 0]

n = int((b - a) / h)
for i in range(n + 1):
    x = a + (h * i)
    corr = y_h[0][i]
    trap = y_h[1][i]
    rung = y_h[2][i]
    adam = y_h[3][i]
    df.loc[len(df.index)] = [i, x, corr, trap, rung, adam]
    if i % 2 == 0:
        err[0] = runge_rule(err[0], y_h[1][i], y_2h[0][i // 2], 2)
        err[1] = runge_rule(err[1], y_h[2][i], y_2h[1][i // 2], 4)
        err[2] = runge_rule(err[2], y_h[3][i], y_2h[2][i // 2], 4)
    
    max_abs_err[0] = max_absolute_error(max_abs_err[0], corr, trap)
    max_abs_err[1] = max_absolute_error(max_abs_err[1], corr, rung)
    max_abs_err[2] = max_absolute_error(max_abs_err[2], corr, adam)
    
abs_err_df.loc[len(abs_err_df.index)] = max_abs_err
err_df.loc[len(err_df.index)] = err

#### Вывод таблиц

In [36]:
df

,i,x,Точное значение,Неявный метод трапеций,Рунге-Кутты 4-го порядка,Адамса 4-го порядка
0,0.0,1.0,0.666667,0.666667,0.666667,0.666667
1,1.0,1.1,0.685358,0.685443,0.685358,0.685358
2,2.0,1.2,0.697674,0.697835,0.697674,0.697674
3,3.0,1.3,0.704607,0.704833,0.704607,0.704607
4,4.0,1.4,0.707071,0.707347,0.707071,0.707070
5,5.0,1.5,0.705882,0.706197,0.705882,0.705881
6,6.0,1.6,0.701754,0.702095,0.701754,0.701754
7,7.0,1.7,0.695297,0.695652,0.695296,0.695296
8,8.0,1.8,0.687023,0.687385,0.687023,0.687023
9,9.0,1.9,0.677362,0.677724,0.677362,0.677362


#### $$ max(|u(x_i) - y_i|)_{(i=0,N)} $$

In [38]:
abs_err_df

,Неявный метод трапеций,Рунге-Кутты 4-го порядка,Адамса 4-го порядка
0,0.000362,1.996768e-07,0.000001


#### Оценка погрешности по правилу Рунге

In [40]:
err_df

,Неявный метод трапеций,Рунге-Кутты 4-го порядка,Адамса 4-го порядка
0,0.000363,2.272593e-07,8.412684e-07


Вывод: Метод Рунге-Кутты показывает себя лучше остальных для данной задачи Коши